In [2]:
:dep ndarray
:dep once_cell
:dep ndarray-rand
:dep rand_isaac

use ndarray::prelude::*;
use ndarray::{concatenate, Axis};
use once_cell::sync::Lazy;
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use rand_isaac::isaac64::Isaac64Rng;
use ndarray_rand::rand::SeedableRng;
use std::any::Any;

In [108]:
const L_SIZE: usize = 100;

static UP_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // left
    let mut arr = [0_usize; L_SIZE];
    for i in 0..L_SIZE - 1 {
        arr[i] = i + 1;
    }
    arr[L_SIZE - 1] = 0;
    arr
});
static DOWN_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // right
    let mut arr = [0_usize; L_SIZE];
    for i in 1..L_SIZE {
        arr[i] = i - 1;
    }
    arr[0] = L_SIZE - 1;
    arr
});

enum Direction {
    Up,
    Down,
    Left,
    Right,
}

fn roll(arr: &Array<f64, Ix2>, direction: Direction) -> Array<f64, Ix2> {
    match direction{
        Direction::Up => {
            concatenate![Axis(0), arr.slice(s![1.., ..]), arr.slice(s![0..1, ..])]
        },
        Direction::Down => {
            concatenate![Axis(0), arr.slice(s![L_SIZE-1.., ..]), arr.slice(s![..L_SIZE-1, ..])]
        },
        Direction::Left => {
            concatenate![Axis(1), arr.slice(s![.., 1..]), arr.slice(s![.., 0..1])]
        },
        Direction::Right => {
            concatenate![Axis(1), arr.slice(s![.., L_SIZE-1..]), arr.slice(s![.., ..L_SIZE-1])]
        },
    }
}

fn nabla2C(c: &Array<f64, Ix2>) -> Array<f64, Ix2> {
    (
        roll(c, Direction::Up) + 
        roll(c, Direction::Down) + 
        roll(c, Direction::Left) + 
        roll(c, Direction::Right) - 
        4. * c
    ) / 4.
}

fn cartesian_product(arr1: &Array<f64, Ix1>, arr2: &Array<f64, Ix1>) -> Array2<f64> {
    let (n, m) = (arr1.len(), arr2.len());
    let mut product = Array2::<f64>::zeros((n * m, 2));

    for (i, val1) in arr1.iter().enumerate() {
        for (j, val2) in arr2.iter().enumerate() {
            product[[i * m + j, 0]] = *val1;
            product[[i * m + j, 1]] = *val2;
        }
    }

    product
}

fn _distance_x(deltaX: &Array<f64, Ix3>) -> Array<f64, Ix2> {
    (deltaX.mapv(|x| x.powi(2)).sum_axis(Axis(2))).mapv(f64::sqrt)
}


pub struct PatternFormationConfig {
    couplingK: f64, alpha: f64, u0: f64, v0: f64, dt: f64, boundaryLength: f64,
    decayRateKd: f64, feedRateKf: f64, productRateKU: f64, productRateKV: f64,
    diffusionRateDU: f64, diffusionRateDV: f64, chemoBetaU: f64, chemoBetaV: f64, 
    agentsNum: usize, cellNumInLine: usize, speedV: f64, randomSeed: u64
}

impl Default for PatternFormationConfig {
    fn default() -> Self {
        PatternFormationConfig{
            couplingK: 1.,
            alpha: 1.,
            u0: 0.012,
            v0: 0.012,
            dt: 0.02,
            boundaryLength: 10.,
            decayRateKd: 1.,
            feedRateKf: 1.,
            productRateKU: 1.,
            productRateKV: 1.,
            diffusionRateDU: 0.01,
            diffusionRateDV: 0.01,
            chemoBetaU: 1.,
            chemoBetaV: 1.,
            agentsNum: 1000,
            cellNumInLine: 250,
            speedV: 3.,
            randomSeed: 10,
        }
    }
}
impl std::fmt::Debug for PatternFormationConfig {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormationConfig")
            .field("couplingK", &self.couplingK)
            .field("alpha", &self.alpha)
            .field("u0", &self.u0)
            .field("v0", &self.v0)
            .field("dt", &self.dt)
            .field("boundaryLength", &self.boundaryLength)
            .field("decayRateKd", &self.decayRateKd)
            .field("feedRateKf", &self.feedRateKf)
            .field("productRateKU", &self.productRateKU)
            .field("productRateKV", &self.productRateKV)
            .field("diffusionRateDU", &self.diffusionRateDU)
            .field("diffusionRateDV", &self.diffusionRateDV)
            .field("chemoBetaU", &self.chemoBetaU)
            .field("chemoBetaV", &self.chemoBetaV)
            .field("agentsNum", &self.agentsNum)
            .field("cellNumInLine", &self.cellNumInLine)
            .field("speedV", &self.speedV)
            .field("randomSeed", &self.randomSeed)
            .finish()
    }
}

pub struct PatternFormation {
    positionX: Array<f64, Ix2>, 
    phaseTheta: Array<f64, Ix1>, freqOmega: Array<f64, Ix1>,
    cemoU: Array<f64, Ix2>, cemoV: Array<f64, Ix2>,
    boundaryLength: f64,
    halfBoundaryLength: f64,
    cellNumInLine: usize,
    cemoPositon: Array<f64, Ix2>,
    dx: f64,
    agentsNum: usize,
    productRateKU: f64,
    productRateKV: f64,
    decayRateKd: f64,
    feedRateKf: f64,
    diffusionRateDU: f64,
    diffusionRateDV: f64,
    chemoBetaU: f64,
    chemoBetaV: f64,
    u0: f64,
    v0: f64,
    dt: f64,
    speedV: f64,
    couplingK: f64,
    alpha: f64,
    config: PatternFormationConfig,
    // tempState
    newAxisPositionX: Array<f64, Ix3>,
    newAxisPhaseTheta: Array<f64, Ix2>,
    tempDistanceCX: Array<f64, Ix2>,
    tempDirectionP: Array<f64, Ix2>,
}


impl PatternFormation {
    fn new(config: PatternFormationConfig) -> Self {
        let mut rng = Isaac64Rng::seed_from_u64(config.randomSeed);
        let positionX = Array::random_using(
            (config.agentsNum, 2), 
            Uniform::new(0., config.boundaryLength), 
            &mut rng
        );
        let phaseTheta = Array::random_using(
            config.agentsNum, Uniform::new(0., 2. * std::f64::consts::PI), &mut rng);
        let halfFreq = Array::random_using(config.agentsNum / 2, Uniform::new(1., 3.), &mut rng);
        let freqOmega = concatenate![
            Axis(0), halfFreq.clone(), -halfFreq.clone()
        ];
        let cemoU = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.u0;
        let cemoV = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.v0;
        let cemoPositon = cartesian_product(
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine),
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine)
        );
        let dx = config.boundaryLength / config.cellNumInLine as f64;
        let halfBoundaryLength = config.boundaryLength / 2.;
        let newAxisPositionX = positionX.clone().insert_axis(Axis(1));
        let newAxisPhaseTheta = phaseTheta.clone().insert_axis(Axis(1));
        let tempDistanceCX = Array::zeros((config.agentsNum, config.cellNumInLine * config.cellNumInLine));
        let tempDirectionP = concatenate![
            Axis(1), 
            phaseTheta.mapv(f64::cos).slice(s![.., NewAxis]), 
            phaseTheta.mapv(f64::sin).slice(s![.., NewAxis])
        ];

        PatternFormation {
            positionX: positionX, 
            phaseTheta: phaseTheta, 
            freqOmega: freqOmega,
            cemoU: cemoU, 
            cemoV: cemoV,
            boundaryLength: config.boundaryLength,
            halfBoundaryLength: halfBoundaryLength,
            cellNumInLine: config.cellNumInLine,
            cemoPositon: cemoPositon,
            dx: dx,
            agentsNum: config.agentsNum,
            productRateKU: config.productRateKU,
            productRateKV: config.productRateKV,
            decayRateKd: config.decayRateKd,
            feedRateKf: config.feedRateKf,
            diffusionRateDU: config.diffusionRateDU,
            diffusionRateDV: config.diffusionRateDV,
            chemoBetaU: config.chemoBetaU,
            chemoBetaV: config.chemoBetaV,
            u0: config.u0,
            v0: config.v0,
            dt: config.dt,
            speedV: config.speedV,
            couplingK: config.couplingK,
            alpha: config.alpha,
            config: config,
            newAxisPositionX: newAxisPositionX,
            newAxisPhaseTheta: newAxisPhaseTheta,
            tempDistanceCX: tempDistanceCX,
            tempDirectionP: tempDirectionP,
        }
    }
    fn _delta_x(&self, position: &Array<f64, Ix2>, others: &Array<f64, Ix3>) -> Array<f64, Ix3> {
        // let subX = position - others;
        (position - others).mapv(|x| {
            if x > self.halfBoundaryLength {
                x - self.boundaryLength
            } else if x < -self.halfBoundaryLength {
                x + self.boundaryLength
            } else {
                x
            }
        })
    } 
    fn _mod_position(&self, position: &Array<f64, Ix2>) -> Array<f64, Ix2> {
        position.mapv(|x| {
            if x > self.boundaryLength {
                x - self.boundaryLength
            } else if x < 0. {
                x + self.boundaryLength
            } else {
                x
            }
        })
    }
    fn directionP(&self) -> Array<f64, Ix2> {
        concatenate![
            Axis(1), 
            self.phaseTheta.mapv(f64::cos).slice(s![.., NewAxis]), 
            self.phaseTheta.mapv(f64::sin).slice(s![.., NewAxis])
        ]
    }
    fn deltaCX(&self) -> Array<f64, Ix3> {
        self._delta_x(&self.cemoPositon, &self.newAxisPositionX)
    }   
    fn deltaX(&self) -> Array<f64, Ix3> {
        self._delta_x(&self.positionX, &self.newAxisPositionX)
    }
    fn adjacency(&self) -> Array<f64, Ix2> {
        (_distance_x(&self.deltaX()) / -self.alpha).mapv(|x| f64::exp(x))
    }
    fn dotTheta(&self) -> Array<f64, Ix1> {
        &self.freqOmega + self.couplingK * (
            self.adjacency() * (&self.newAxisPhaseTheta - &self.phaseTheta)
        ).sum_axis(Axis(0))
    }
    fn update(&mut self) {
        self.newAxisPositionX.assign(&self.positionX.clone().insert_axis(Axis(1)));
        self.newAxisPhaseTheta.assign(&self.phaseTheta.clone().insert_axis(Axis(1)));
        self.tempDistanceCX.assign(&_distance_x(&self.deltaCX()));
        self.tempDirectionP.assign(&self.directionP());
        self.positionX.assign(&(self.dt * self.speedV * &self.tempDirectionP));
        self.positionX.assign(&self._mod_position(&self.positionX));
    }
}
impl std::fmt::Debug for PatternFormation {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormation on")
            .field("", &self.config)
            .finish()
    }
}


let c: Array<f64, Ix2> = Array::zeros((L_SIZE, L_SIZE));
// let c: Array<f64, Ix2> = Array::from_shape_vec((3, 3), vec![1., 2., 3., 4., 5., 6., 7., 8., 9.]).unwrap();

In [115]:
// let config = PatternFormationConfig::default();
let mut model = PatternFormation::new(PatternFormationConfig {
    cellNumInLine: 100,
    ..Default::default() // 使用其他默认值
});

// let others = model.positionX.clone().insert_axis(Axis(1));
// let subX = model.positionX.clone() - others;
// model._delta_x(&model.positionX, &model.positionX.clone().insert_axis(Axis(1)))

In [114]:
model.dotTheta()

[-65.68208292889942, 127.13880680764652, 8.839083901147328, 107.053735148406, 108.45889617554754, ..., 3.3192618651834316, -85.77975504243184, 123.20568491944722, -13.181240663328468, -92.84579396764052], shape=[1000], strides=[1], layout=CFcf (0xf), const ndim=1

In [119]:
use std::time::{Instant};
let counts: usize = 100;
fn func(model: &PatternFormation) {
    // c.select(Axis(0), &*UP_ROLL_SLC);
    model.dotTheta();
    // concatenate![Axis(0), c.slice(s![1.., ..]), c.slice(s![0..1, ..])];
}

let start = Instant::now();
func(&model);
let mut duration = start.elapsed();
for _ in 0..(counts - 1) {
    let start = Instant::now();
    func(&model);
    duration += start.elapsed();
}
println!("mean duration: {:?}", duration / counts as u32);

mean duration: 53.202678ms


In [130]:
c.select(Axis(1), &*DOWN_ROLL_SLC2)

[[2.0, 3.0, 1.0],
 [5.0, 6.0, 4.0],
 [8.0, 9.0, 7.0]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2

In [22]:
testC

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

In [131]:
// 实现类似numpy.roll的功能


// roll(&testC, 1)

In [137]:
roll(&c, Direction::Right)

[[3.0, 1.0, 2.0],
 [6.0, 4.0, 5.0],
 [9.0, 7.0, 8.0]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2